In [1]:
# import necessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
# import for headless browsing
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless")



In [3]:
def scrape(url):
    '''Function to scrape the reviews from the website'''
    # Open the browser
    driver = webdriver.Chrome(options=chrome_options)
    # Open the URL
    driver.get(url)
    # Wait for the page to load
    time.sleep(10)
    # Close the pop-up if it appears
    try:
        # Wait for the pop-up to appear and the exit button to be clickable
        #wait = WebDriverWait(driver, 10)
        # find the exit button
        exit_button_xpath = '//*[@id="ltkpopup-content"]/div[3]/button'
        exit_button = driver.find_element(By.XPATH, exit_button_xpath)

        # Click the exit button
        exit_button.click()
        print("Pop-up closed.")
        # Wait for the pop-up to close
    except TimeoutException:
        # Handle cases where the pop-up does not appear within the timeout
        print("No pop-up appeared within the timeout period.")
    # html object for the entire page
    html = driver.find_element(By.TAG_NAME, 'html')
    # Scroll to the bottom of the page
    for _ in range(10):  # Adjust the range depending on how much you need to scroll
        # Scroll down to the bottom of the page
        html.send_keys(Keys.END)
    # find the review date
    date_element = driver.find_elements(By.XPATH, "//div[2]/div/div/div/div[@class='time-stamp']")
    # Extract the text from the date element
    dates = [date.text for date in date_element]
    # find the review title
    title_element = driver.find_elements(By.XPATH, "//div[2]/div/div/div/h3")
    # Extract the text from the title element
    titles = [title.text for title in title_element]
    # find the review text
    reviews_element = driver.find_elements(By.XPATH, "//div[2]/div/div/div[@class='ls-wrapper col-10-10 last margin-top']/p")
    # Extract the text from the review element
    reviews = [review.text for review in reviews_element]
    # Close the browser
    driver.quit()
    # Return the dates, titles, and reviews
    return dates, titles, reviews

In [15]:
# URL of the website to scrape
url = "https://www.lovelyskin.com/o/reviews/la-roche-posay-toleriane-double-repair-moisturizer"

# initialize lists to store the reviews, titles, and dates
all_reviews = []
all_titles = []
all_dates = []

# Scrape the first page
dates, titles, reviews = scrape(url)
all_reviews.append(reviews)
all_titles.append(titles)
all_dates.append(dates)

# Scrape the remaining pages
add_url = "?page="

# Loop through the remaining pages
for x in range(2, 24):
    # Construct the URL for the page
    url = f"https://www.lovelyskin.com/o/reviews/la-roche-posay-toleriane-double-repair-moisturizer{add_url}{x}"
    # Scrape the page
    dates, titles, reviews = scrape(url)
    all_reviews.append(reviews)
    all_titles.append(titles)
    all_dates.append(dates)

Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.
Pop-up closed.


In [16]:
# https://stackoverflow.com/questions/952914/how-do-i-make-a-flat-list-out-of-a-list-of-lists
# Flatten the lists
all_reviews = [item for sublist in all_reviews for item in sublist]
all_titles = [item for sublist in all_titles for item in sublist]
all_dates = [item for sublist in all_dates for item in sublist]

In [17]:
# Create a DataFrame for the reviews
df = pd.DataFrame(list(zip(all_dates, all_titles, all_reviews)), columns =['Date', 'Title', 'Review'])
print(len(df))
df.head()


448


,Date,Title,Review
0,"April 13, 2024",Mi crema favorita,It's the best cream I've had in years!
1,"April 11, 2024",My Go To!,This is my go to daily morning moisturizer! It...
2,"April 10, 2024",Great,My daughter uses the cream day and night and s...
3,"April 06, 2024",very hydrating,I use this mixed with a small amount of rosehi...
4,"April 06, 2024",Irritation,I started using this moisturizer and another s...


In [18]:
# Save the DataFrame to a CSV file
df.to_csv('reviews.csv', index=False)